# Demonstration of linear mosaic

Reads a CASA MeasurementSet containing 7 pointing observation of CASA at 8.4 GHz, forms linear mosaic of dirty images

In [ ]:
% matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.data.data_models import Visibility
from arl.data.polarisation import PolarisationFrame

from arl.visibility.base import create_visibility, create_visibility_from_ms
from arl.skycomponent.operations import create_skycomponent
from arl.image.operations import show_image, copy_image
from arl.image.deconvolution import deconvolve_cube
from arl.image.iterators import image_raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration
from arl.util.primary_beams import create_pb_vla
from arl.imaging import create_image_from_visibility, invert_2d
from arl.imaging.weighting import weight_visibility

from arl.data.polarisation import PolarisationFrame

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))


In [ ]:
vis_list=create_visibility_from_ms('../../data/vis/xcasa.ms')

In [ ]:
# plot the amplitude of the visibility for each field
for field, vt in enumerate(vis_list):
    uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
    plt.clf()
    plt.plot(uvdist, numpy.abs(vt.data['vis']), '.')
    plt.xlabel('uvdist')
    plt.ylabel('Amp Visibility')
    plt.title('Field %d' % (field))
    plt.show()

Make the dirty images

In [ ]:
cellsize=0.00001

model = create_image_from_visibility(vt, cellsize=cellsize, npixel=512, nchan=2, 
                                     phasecentre=vis_list[0].phasecentre,
                                     polarisation_frame=PolarisationFrame('stokesIQUV'))
mosaic = copy_image(model)
mosaicsens = copy_image(model)
work = copy_image(model)

for vt in vis_list:
    beam = create_pb_vla(model, pointingcentre=vt.phasecentre)
    beam.data /= numpy.max(beam.data)
    dirty, sumwt = invert_2d(vt, model)
    mosaic.data += dirty.data * beam.data
    mosaicsens.data += beam.data**2
    
show_image(mosaic, title='Linear mosaic')
plt.show()
show_image(mosaicsens, title='Linear mosaic sensitivity')
plt.show()
